<a href="https://colab.research.google.com/github/brucker3/KDM_spring2021/blob/main/icp4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# downloading java for spark 
!sudo apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# downloading  and set up spark 
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,391 kB]
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRel

In [4]:
# set up env 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

import findspark
findspark.init()

In [5]:
# find spark 
findspark.find()


'/content/spark-3.0.1-bin-hadoop2.7'

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("icp4")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [7]:
#spark check that its set up correctly 
spark

In [9]:
# loading data set 

df = spark.read.csv("/content/data.csv", header=True, inferSchema= True)
df.printSchema()


root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



In [10]:
df.show(5, truncate=False)

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+-------------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines   |InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract      |PaperlessBilling|PaymentMethod            |MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+-------------------------+--------------+------------+-----+
|7590-VHVEG|Female|0            |Yes    |No        |1     |No          |No phone service|DSL            |No            |Yes         |No              |N

Creat new data set filter why its important 
Transformation vs action 
look at tuesday lecture for action vs transformation 

In [17]:
# transforamiton  and action 1 
# so in total there are 7043  after filtering for tenure the count is 5073
df_1 =df.select('gender','seniorCitizen')
df_1 = df.filter(df.tenure > 10)
df_1.show()
df_1.count()
df.count()

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|5575-GNVDE|  Male|            0|     No|        No|    34|  

7043

In [18]:

from pyspark.sql.functions import sum
df_2 =df.select('SeniorCitizen','tenure', 'MonthlyCharges')
df_2.show(5)
print(df_2.groupBy().sum().collect())
# transformition and action 2 removed non quantative rows and summed them 
# some intering finding TotalCharges is not consired a quanttative row 
# so only three on 20 rows are 
# also of the 7043 only 1142 are senior for 16.2%

+-------------+------+--------------+
|SeniorCitizen|tenure|MonthlyCharges|
+-------------+------+--------------+
|            0|     1|         29.85|
|            0|    34|         56.95|
|            0|     2|         53.85|
|            0|    45|          42.3|
|            0|     2|          70.7|
+-------------+------+--------------+
only showing top 5 rows

[Row(sum(SeniorCitizen)=1142, sum(tenure)=227990, sum(MonthlyCharges)=456116.6000000003)]
[Row(avg(SeniorCitizen)=0.1621468124378816, avg(tenure)=32.37114865824223, avg(MonthlyCharges)=64.76169246059922)]


In [23]:



# action and transfomriont 3 
# based on mean of monthly charges 
print(df.groupBy().mean().collect())

df_3 = df.filter(df.MonthlyCharges > 64)
df_3.show(5) # new df looks good 
# loook on the how the number changes once you take the hight side of the ave of monthly payments
print(df_3.groupBy().sum().collect())
print(df_3.groupBy().mean().collect())



[Row(avg(SeniorCitizen)=0.1621468124378816, avg(tenure)=32.37114865824223, avg(MonthlyCharges)=64.76169246059922)]
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|9237-HQITU|Female|            0|     No|        No|     2| 